In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from CPVDataset import CPVDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator,EmbeddingClassifier
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [3]:
label_column = "Omschrijving"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")
input_df2=pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_foreign_ted1.csv")
all_input=pd.concat([input_df,input_df2])

In [4]:
vc=all_input[label_column].value_counts()

In [5]:
running_device = torch.device("cuda")

In [15]:
prd = ProcessData(all_input, label_column, text_column, sample_num=100, aug_limit=100,test_per_label_num=30, drop_limit=50,
                  balanced_test=True)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
# balanced_test=test_df.sample(frac=1).groupby(label_column, sort=False).head(30)
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 263700
After drop cpv with low observations we have : 237451
The num of class is  1055
The number of total data is:  96297
Total number of test data is:  31650
Total number of train data is:  61886


In [16]:
value_counts

Apotheekdiensten                          70
Bouwen van hoofdweg                       70
Politieauto's                             70
Massaspectrometers                        70
Speciale voertuigen                       70
                                          ..
Fotokopieer- en thermokopieerapparaten    18
Zout en zuiver natriumchloride            18
Accommodatiediensten                      18
Uitkeringsdiensten                        17
Periodieken                               17
Name: Omschrijving, Length: 1055, dtype: int64

In [17]:
from joblib import dump, load
dump(onehot, '/content/drive/MyDrive/gan_bert_for_cpv/output/description/onehot.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/description/onehot.joblib']

In [18]:
dump(label_encoder, '/content/drive/MyDrive/gan_bert_for_cpv/output/description/labelencoder.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/description/labelencoder.joblib']

In [19]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [ ]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 0, running_device, max_length=100)
suf_dataset=cd.create(insuf_df)
test_dataset=cd.create(test_df)

Init new bert


Getting bert hidden states from layer: 0: 100%|██████████| 31650/31650 [06:49<00:00, 77.36it/s]


In [ ]:
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
train_dl = DataLoader(suf_dataset, shuffle=True, batch_size=20)

In [ ]:
from models import HiddenClassifier9
classifier = HiddenClassifier9(prd.num_class, bert)
classifier = model_runner.train_classifier(train_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

In [ ]:
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=2e-5, n_epochs=3)

In [ ]:
aug_limits = decide_aug_labels(True, 800, value_counts,1)

In [ ]:
augmented_ds_lists = model_runner.augment_data(bert, aug_limits, insuf_ds_dict,batch_size=40, g_epochs=10, glr=2e-6, dlr=2e-6)

We have: 980 labels to be augmented


  0%|          | 0/980 [00:58<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
gan_aug_train = torch.utils.data.ConcatDataset([original_train_dataset]+augmented_ds_lists)
gan_aug_dl = DataLoader(gan_aug_train, shuffle=True, batch_size=40)

In [ ]:
classifier = EmbeddingClassifier(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)